<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/custom_RAG_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install PyPDF2 transformers sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.5 MB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() or "" # Add empty string for pages with no text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None
    return text

# Replace with the actual path to your uploaded PDF file
pdf_path = "/content/The Public Domain_ Enclosing the Commons of the Mind.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

if pdf_text:
    print(f"Successfully extracted text from {pdf_path}. Length: {len(pdf_text)}")
else:
    print(f"Failed to extract text from {pdf_path}")

Successfully extracted text from /content/The Public Domain_ Enclosing the Commons of the Mind.pdf. Length: 920436


In [4]:
%pip install pytesseract

In [3]:
from transformers import AutoTokenizer

def split_text_into_chunks(text, chunk_size=512, overlap=50):
    """Splits text into chunks with a specified size and overlap."""
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # Using a common tokenizer
    # Add max_length and truncation to prevent the warning
    tokens = tokenizer.encode(text, add_special_tokens=False, max_length=tokenizer.model_max_length, truncation=True)

    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i : i + chunk_size]
        chunks.append(tokenizer.decode(chunk_tokens))
    return chunks

if pdf_text:
    text_chunks = split_text_into_chunks(pdf_text)
    print(f"Split text into {len(text_chunks)} chunks.")
    # Optional: Display the first chunk
    if text_chunks:
        print("\nFirst chunk:")
        print(text_chunks[0])
else:
    text_chunks = []
    print("No text to split into chunks.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Split text into 2 chunks.

First chunk:
the public domain _ _ _ - 1 _ _ _ 0 _ _ _ 137278 _ u00. qxd 8 / 28 / 08 11 : 04 am page i - 1 _ _ _ 0 _ _ _ 1 _ _ _ thomas jefferson to isaac mcpherson, august 13, 1813, p. 6. 37278 _ u00. qxd 8 / 28 / 08 11 : 04 am page iijames boyle the public domain enclosing the commons of the mind yale university press new haven & london _ _ _ - 1 _ _ _ 0 _ _ _ 137278 _ u00. qxd 8 / 28 / 08 11 : 04 am page iiia caravan book. for more information, visit www. caravanbooks. org. copyright © 2008 by james boyle. all rights reserved. the author has made an online version of this work available under a creative commons attribution - noncommercial - share alike 3. 0 license. it can be accessedthrough the author ’ s website at http : / / james - boyle. com. printed in the united states of america. isbn : 978 - 0 - 300 - 13740 - 8 library of congress control number : 2008932282 a catalogue record for this book is available from the british library. this paper meets t

In [5]:
from sentence_transformers import SentenceTransformer

def create_embeddings(text_chunks):
    """Creates embeddings for text chunks using a pre-trained model."""
    model = SentenceTransformer('all-MiniLM-L6-v2') # Using a common sentence embedding model
    embeddings = model.encode(text_chunks, show_progress_bar=True)
    return embeddings

if text_chunks:
    chunk_embeddings = create_embeddings(text_chunks)
    print(f"Created embeddings for {len(chunk_embeddings)} chunks.")
    # Optional: Display the shape of the embeddings
    # if chunk_embeddings.shape:
    #     print("\nShape of embeddings:")
    #     print(chunk_embeddings.shape)
else:
    chunk_embeddings = None
    print("No chunks to create embeddings for.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Created embeddings for 2 chunks.


In [6]:
import faiss
import numpy as np

def create_faiss_index(embeddings):
    """Creates a FAISS index from embeddings."""
    if embeddings is None or len(embeddings) == 0:
        print("No embeddings to create an index from.")
        return None

    # Ensure embeddings are in the correct format (numpy array of float32)
    embeddings = np.array(embeddings).astype('float32')

    # Get the dimension of the embeddings
    embedding_dimension = embeddings.shape[1]

    # Create an index (e.g., IndexFlatL2 for L2 distance)
    index = faiss.IndexFlatL2(embedding_dimension)

    # Add the embeddings to the index
    index.add(embeddings)

    print(f"Created FAISS index with {index.ntotal} embeddings.")
    return index

if chunk_embeddings is not None:
    faiss_index = create_faiss_index(chunk_embeddings)
else:
    faiss_index = None
    print("No embeddings available to create FAISS index.")

Created FAISS index with 2 embeddings.


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def load_llm(model_name="distilgpt2"):
    """Loads an open-source LLM and its tokenizer."""
    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Load model
        model = AutoModelForCausalLM.from_pretrained(model_name)
        print(f"Successfully loaded model: {model_name}")
        return tokenizer, model
    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        print("Please ensure you have access to this model or try a different one.")
        return None, None

# Replace with your preferred open-source LLM
# Note: Some models may require authentication or have specific hardware requirements
llm_tokenizer, llm_model = load_llm()

if llm_model and llm_tokenizer:
    print("LLM and tokenizer loaded successfully.")
else:
    print("Failed to load LLM and tokenizer.")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Successfully loaded model: distilgpt2
LLM and tokenizer loaded successfully.


In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def retrieve_relevant_chunks(query, faiss_index, text_chunks, embedding_model, top_k=5):
    """Retrieves the most relevant text chunks for a given query using the FAISS index."""
    # Create embedding for the query using the provided embedding model
    query_embedding = embedding_model.encode([query]).astype('float32')

    # Perform similarity search
    distances, indices = faiss_index.search(query_embedding, top_k)

    # Get the relevant text chunks, ensuring indices are within bounds
    relevant_chunks = [text_chunks[i] for i in indices[0] if i < len(text_chunks)]

    return relevant_chunks

In [9]:
def generate_response(query, relevant_chunks, llm_tokenizer, llm_model):
    """Generates a response using the LLM based on the query and relevant chunks."""
    # Combine relevant chunks into a single context
    context = "\n\n".join(relevant_chunks)

    # Create a prompt for the LLM
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"

    # Tokenize the prompt and ensure padding
    llm_tokenizer.pad_token = llm_tokenizer.eos_token # Set pad token to eos token
    inputs = llm_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")

    # Generate response from the LLM
    try:
        output = llm_model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=200, num_return_sequences=1, no_repeat_ngram_size=2)
        response = llm_tokenizer.decode(output[0], skip_special_tokens=True)
        # Simple post-processing to remove the original prompt from the response
        response = response.replace(prompt, "").strip()
        return response
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Sorry, I could not generate a response."

# Example usage (assuming you have a query and the necessary components from previous steps)
# query = "What is the main topic of the book?"
# if faiss_index and text_chunks and chunk_embeddings is not None and llm_model and llm_tokenizer:
#      retrieved_chunks = retrieve_relevant_chunks(query, faiss_index, text_chunks, SentenceTransformer('all-MiniLM-L6-v2'))
#      answer = generate_response(query, retrieved_chunks, llm_tokenizer, llm_model)
#      print("\nAnswer:")
#      print(answer)
# else:
#      print("Cannot generate response. Ensure all components (index, chunks, embeddings, LLM) are available.")

In [11]:
import os

# Install poppler-utils for pdf2image
!apt-get install -y poppler-utils

# Install pdf2image
%pip install pdf2image

print("Poppler-utils and pdf2image installed.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 0s (466 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
Poppler-utils and pdf2image installed.


In [12]:
from PyPDF2 import PdfReader
from PIL import Image
import pytesseract
from pdf2image import convert_from_path

def extract_text_from_pdf_with_ocr_fallback(pdf_path):
    """Extracts text from a PDF file, using OCR as a fallback for pages with no selectable text."""
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for i, page in enumerate(reader.pages):
                page_text = page.extract_text()
                if page_text and page_text.strip(): # Check if text extraction yielded something meaningful
                    text += page_text
                else:
                    # Fallback to OCR if no text was extracted
                    print(f"Performing OCR on page {i+1}...")
                    # Convert PDF page to image for OCR
                    # Note: You might need to specify poppler_path if it's not in your PATH
                    images = convert_from_path(pdf_path, first_page=i+1, last_page=i+1)
                    if images:
                        # Apply OCR to the image
                        ocr_text = pytesseract.image_to_string(images[0])
                        text += ocr_text
    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None
    return text

# Replace with the actual path to your uploaded PDF file
pdf_path = "/content/The Public Domain_ Enclosing the Commons of the Mind.pdf"
pdf_text = extract_text_from_pdf_with_ocr_fallback(pdf_path)

if pdf_text:
    print(f"Successfully extracted text from {pdf_path}. Length: {len(pdf_text)}")
else:
    print(f"Failed to extract text from {pdf_path}")

Performing OCR on page 1...
Successfully extracted text from /content/The Public Domain_ Enclosing the Commons of the Mind.pdf. Length: 920489


In [13]:
from transformers import AutoTokenizer

def split_text_into_chunks(text, chunk_size=512, overlap=50):
    """Splits text into chunks with a specified size and overlap."""
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # Using a common tokenizer
    # Add max_length and truncation to prevent the warning
    tokens = tokenizer.encode(text, add_special_tokens=False, max_length=tokenizer.model_max_length, truncation=True)

    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i : i + chunk_size]
        chunks.append(tokenizer.decode(chunk_tokens))
    return chunks

if pdf_text:
    text_chunks = split_text_into_chunks(pdf_text)
    print(f"Split text into {len(text_chunks)} chunks.")
    # Optional: Display the first chunk
    if text_chunks:
        print("\nFirst chunk:")
        print(text_chunks[0])
else:
    text_chunks = []
    print("No text to split into chunks.")

Split text into 2 chunks.

First chunk:
the public enue ao au muu em utils james boyle the public domain _ _ _ - 1 _ _ _ 0 _ _ _ 137278 _ u00. qxd 8 / 28 / 08 11 : 04 am page i - 1 _ _ _ 0 _ _ _ 1 _ _ _ thomas jefferson to isaac mcpherson, august 13, 1813, p. 6. 37278 _ u00. qxd 8 / 28 / 08 11 : 04 am page iijames boyle the public domain enclosing the commons of the mind yale university press new haven & london _ _ _ - 1 _ _ _ 0 _ _ _ 137278 _ u00. qxd 8 / 28 / 08 11 : 04 am page iiia caravan book. for more information, visit www. caravanbooks. org. copyright © 2008 by james boyle. all rights reserved. the author has made an online version of this work available under a creative commons attribution - noncommercial - share alike 3. 0 license. it can be accessedthrough the author ’ s website at http : / / james - boyle. com. printed in the united states of america. isbn : 978 - 0 - 300 - 13740 - 8 library of congress control number : 2008932282 a catalogue record for this book is availab

In [14]:
from sentence_transformers import SentenceTransformer

def create_embeddings(text_chunks):
    """Creates embeddings for text chunks using a pre-trained model."""
    model = SentenceTransformer('all-MiniLM-L6-v2') # Using a common sentence embedding model
    embeddings = model.encode(text_chunks, show_progress_bar=True)
    return embeddings

if text_chunks:
    chunk_embeddings = create_embeddings(text_chunks)
    print(f"Created embeddings for {len(chunk_embeddings)} chunks.")
    # Optional: Display the shape of the embeddings
    # if chunk_embeddings.shape:
    #     print("\nShape of embeddings:")
    #     print(chunk_embeddings.shape)
else:
    chunk_embeddings = None
    print("No chunks to create embeddings for.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Created embeddings for 2 chunks.


In [15]:
import faiss
import numpy as np

def create_faiss_index(embeddings):
    """Creates a FAISS index from embeddings."""
    if embeddings is None or len(embeddings) == 0:
        print("No embeddings to create an index from.")
        return None

    # Ensure embeddings are in the correct format (numpy array of float32)
    embeddings = np.array(embeddings).astype('float32')

    # Get the dimension of the embeddings
    embedding_dimension = embeddings.shape[1]

    # Create an index (e.g., IndexFlatL2 for L2 distance)
    index = faiss.IndexFlatL2(embedding_dimension)

    # Add the embeddings to the index
    index.add(embeddings)

    print(f"Created FAISS index with {index.ntotal} embeddings.")
    return index

if chunk_embeddings is not None:
    faiss_index = create_faiss_index(chunk_embeddings)
else:
    faiss_index = None
    print("No embeddings available to create FAISS index.")

Created FAISS index with 2 embeddings.


In [16]:
from sentence_transformers import SentenceTransformer

# Load the embedding model once
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chatbot loop
print("Simple RAG Chatbot (Type 'quit' to exit)")

while True:
    query = input("Enter your query: ")
    if query.lower() == 'quit':
        break

    if faiss_index is not None and text_chunks and chunk_embeddings is not None and llm_model and llm_tokenizer:
        # Retrieve relevant chunks, passing the loaded embedding model
        retrieved_chunks = retrieve_relevant_chunks(query, faiss_index, text_chunks, embedding_model)

        # Generate response
        answer = generate_response(query, retrieved_chunks, llm_tokenizer, llm_model)

        print("\nAnswer:")
        print(answer)
    else:
        print("RAG components not fully initialized. Please run the previous steps.")

Simple RAG Chatbot (Type 'quit' to exit)
Enter your query: what are the contents?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Answer:
The contents of a book are not necessarily the same as the content of an article.
The content is not the exact same. The content contains a number of elements, including the title, title and title. It is the only content that is in a single page. This is because the book contains the information that the reader has already read. If the user has read the first page, the second page will be the subject of discussion. In the case of any other page the page is subject to discussion, and the third page may be subject only to the discussion of that page and not to any discussion about that particular page or the other pages. For example, if the person who read this page has been reading the last page of it, then the next page should be a subject. However, in this case the topic of debate is a topic that has not been discussed. Therefore, it is important to note that this is only the beginning of what is called the “ ‘ ”


KeyboardInterrupt: Interrupted by user